In [1]:
import cv2 as cv
import numpy as np
import io, random
from PIL import Image
from text_detector import get_image_elem 

def display(mat, name=None):
    if name is None:
        name = random.randint(1000, 9999)
        name = str(int(name))
    cv.namedWindow(name)
    cv.imshow(name, mat)
    return name
    # cv.waitKey()
    # try:
    #     cv.destroyWindow('test')
    # except:
    #     pass

ImportError: cannot import name 'get_images_elem' from 'text_detector' (c:\CodeForClass\imgtool\backend\func\text_detector.py)

In [2]:
TEST_IMG = 'test7.jpg'

im = cv.imread(TEST_IMG)
# im = cv.resize(im, (700, 960))

edges = cv.Canny(im, 100, 200)
cv.imwrite('test1_edge.jpg', edges)

contours, _ = cv.findContours(edges, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
print(len(contours))

new_im = cv.drawContours(im.copy(), contours, -1, (255,255,0), 2)

# display(im, 'source')
display(edges, 'edge')
display(new_im, 'contours')

cv.waitKey()
cv.destroyAllWindows()

4390


In [3]:
# im_b = Image.open(TEST_IMG)
# b = io.BytesIO()
# im_b.save(b, 'jpeg')
bboxs = get_images_elem(im)
print(bboxs)
# display(im)

Looking for C:\Users\Joel\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\Joel\.keras-ocr\crnn_kurapan.h5


ValueError: operands could not be broadcast together with shapes (1500,6) (3,) (1500,6) 

In [13]:
# crop an area two times larger than bbox
bbox = bboxs[0]

mask = np.zeros_like(im)

MAX_HEIGHT = im.shape[0]
MAX_WIDTH = im.shape[1]
l = int(float(bbox['left'])); w = int(float(bbox['width'])); 
t = int(float(bbox['top'])); h = int(float(bbox['height']))
rect = ((l, t), (l + w, t + h))
print(rect)

# cv.rectangle(mask, rect[0], rect[1], (255, 255, 255), -1)
# display(mask)
# cv.waitKey()
# cv.destroyAllWindows()

((402, 76), (592, 105))


In [14]:
def get_rotation_matrix(angle):
    angle_in_radians = np.deg2rad(angle)
    c, s = np.cos(angle_in_radians), np.sin(angle_in_radians)
    return np.array([[c, -s], [s, c]]), np.array([[c, s], [-s, c]])

def preprocess(im, textbox):
    '''
        input: image, textbox
        output: area near textbox, rotated
    '''
    image_center = (int(float(textbox['left'])), int(float(textbox['top'])))
    br_corner = (image_center[0] + int(float(textbox['width'])), image_center[1] + int(float(textbox['height'])))
    angle = float(textbox['angle'])

    blk = np.zeros(im.shape, np.uint8)
    cv.rectangle(blk, image_center, br_corner, (0,255,0, 0.5), -1)
    result = cv.addWeighted(im, 1.0, blk, 0.5, 1)

    rot_mat = cv.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv.warpAffine(result, rot_mat, im.shape[1::-1], flags=cv.INTER_LINEAR)
    display(result)
    cv.waitKey()
    cv.destroyAllWindows()
    return result

im_r = cv.imread(TEST_IMG)
new_im_r = preprocess(im_r, bboxs[0])
